In [1]:
import numpy as np
from pathlib import Path
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from PIL import Image
from tensorflow import keras
from collections import Counter

2022-12-08 16:48:57.334098: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-08 16:48:57.334173: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
person_path = Path("../Images_for_Training")
label_names = [person.name for person in person_path.iterdir()]

In [3]:
le = LabelEncoder()
le.fit(label_names)
print(le.classes_)
le.transform(label_names)

['Maria' 'Nikhil' 'Simon']


array([0, 2, 1])

In [4]:
def _565_to_channels(data: np.ndarray) -> np.ndarray:
    width, height = data.shape
    channels = np.zeros(shape=(width, height, 3), dtype=np.uint16)

    for x in range(width):
        for y in range(height):
            pixel = data[x, y]
            r = (pixel & 0xF800) >> 11 # F800: bits 11-15
            g = (pixel & 0x07E0) >> 5  # 07E0: bits 5-10
            b = (pixel & 0x1F)         # 001F: bits 0-4
            channels[x, y, 0] = r
            channels[x, y, 1] = g
            channels[x, y, 2] = b

    return channels


def _565_to_888(data: np.ndarray) -> Image:
    width, height = data.shape
    img = Image.new('RGB', (width, height))

    for x in range(width):
        for y in range(height):
            pixel = data[x, y]
            r = (pixel & 0xF800) >> 11 # F800: bits 11-15
            g = (pixel & 0x07E0) >> 5  # 07E0: bits 5-10
            b = (pixel & 0x1F)         # 001F: bits 0-4
            img.putpixel((x, y), (r * 8, g * 4, b * 8)) # multiply by (8, 4, 8) to scale back to [0,255]

    return img

def _888_to_565(im: Image) -> np.ndarray:
    width, height = im.size
    new_pxs = np.zeros(shape=(width, height), dtype=np.uint16)

    for x in range(width):
        for y in range(height):
            r, g, b = im.getpixel((x,y))

            # downsample to RGB565
            r //= 8
            g //= 4
            b //= 8
            new_pxs[x, y] = (r << 11) | (g << 5) | b

    return new_pxs

In [5]:
labels = []
data = []
for person in person_path.iterdir():
    for file in tqdm(person.iterdir()):
        if str(file).endswith(".npy"):
            img_np = np.load(file)

            img = _565_to_888(img_np)
            img = img.resize((120, 160))  ## scale to lowest res
            
            img_np = _888_to_565(img) # go back to 565, as Arduino will use this
            #print(np.shape(img_np))
            #plt.imshow(img_np)
            img_np = _565_to_channels(img_np)
            #print(img_np.shape)
            #plt.imshow(img_np)

            data.append(img_np)
            labels.append(le.transform([person.name])[0])

labels = np.array(labels)
data = np.array(data)


#w = 10
#h = 10
#fig = plt.figure(figsize=(8, 8))
#columns = 4
#rows = 5
#for i in range(1, columns*rows +1):
#    img = data[np.random.randint(len(data))]
#    fig.add_subplot(rows, columns, i)
#    plt.imshow(img)
#plt.show()

90it [00:19,  4.61it/s]
98it [00:20,  4.69it/s]
72it [00:14,  4.92it/s]


In [6]:
print(labels.shape)
print(data.shape)

(130,)
(130, 120, 160, 3)


In [7]:
print(Counter(labels))

Counter({2: 49, 0: 45, 1: 36})


Save the transformed data as ready to use tensor and labels, with classes that can be loaded to labelEncoder

In [8]:
with open("../Images_Transformed_565_160_120/data.npy", "wb") as f:
    np.save(f, data)

In [9]:
with open("../Images_Transformed_565_160_120/labels.npy", "wb") as f:
    np.save(f, labels)

In [10]:
with open("../Images_Transformed_565_160_120/classes.npy", "wb") as f:
    np.save(f, le.classes_)